In [129]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
hacktivist_messages = pd.read_csv("data/hacktivist_messages.csv", sep=";")
hacktivist_messages = hacktivist_messages.dropna()

In [133]:
from flair.data import Sentence
from flair.models import SequenceTagger
import math

# load tagger
tagger = SequenceTagger.load("flair/ner-english-fast")

tagged_texts = []
for row in tqdm(hacktivist_messages.iterrows(), total=hacktivist_messages.shape[0]):
    text = Sentence(row[1]["Text"])
    tagger.predict(text)
    tagged_texts.append(text)

2024-09-04 12:51:36,393 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


 77%|███████▋  | 2319/2995 [12:12<07:19,  1.54it/s]

In [ ]:
hacktivist_messages["NER"] = tagged_texts